# Eden AI

Eden AI 通过整合顶尖的 AI 供应商，正在革新 AI 领域，赋能用户解锁无限可能，挖掘人工智能的真正潜力。它提供了一个集成化、全面且无忧的平台，让用户能够闪电般地将 AI 功能部署到生产环境中，通过单一 API 轻松访问所有广泛的 AI 能力。 (网站：https://edenai.co/)

本示例将介绍如何使用 LangChain 与 Eden AI 模型进行交互

访问 EDENAI 的 API 需要一个 API 密钥，

你可以通过创建账户 https://app.edenai.run/user/register 并前往 https://app.edenai.run/admin/account/settings 获取。

获得密钥后，我们需要将其设置为环境变量，运行以下命令：

```bash
export EDENAI_API_KEY="..."
```

如果您不希望设置环境变量，可以直接通过 `edenai_api_key` 命名参数传递密钥，在初始化 EdenAI LLM 类时：

In [ ]:
from langchain_community.llms import EdenAI

In [ ]:
llm = EdenAI(edenai_api_key="...", provider="openai", temperature=0.2, max_tokens=250)

## 调用模型

EdenAI API 汇集了各种提供商，每个提供商都提供多种模型。

要访问特定模型，您只需在实例化时添加 'model' 即可。

例如，让我们探索 OpenAI 提供的模型，例如 GPT3.5

### 文本生成

In [ ]:
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate

llm = EdenAI(
    feature="text",
    provider="openai",
    model="gpt-3.5-turbo-instruct",
    temperature=0.2,
    max_tokens=250,
)

prompt = """
User: Answer the following yes/no question by reasoning step by step. Can a dog drive a car?
Assistant:
"""

llm(prompt)

### 图像生成

In [ ]:
import base64
from io import BytesIO

from PIL import Image


def print_base64_image(base64_string):
    # Decode the base64 string into binary data
    decoded_data = base64.b64decode(base64_string)

    # Create an in-memory stream to read the binary data
    image_stream = BytesIO(decoded_data)

    # Open the image using PIL
    image = Image.open(image_stream)

    # Display the image
    image.show()

In [ ]:
text2image = EdenAI(feature="image", provider="openai", resolution="512x512")

In [ ]:
image_output = text2image("A cat riding a motorcycle by Picasso")

In [ ]:
print_base64_image(image_output)

### 使用回调进行文本生成

In [ ]:
from langchain_community.llms import EdenAI
from langchain_core.callbacks import StreamingStdOutCallbackHandler

llm = EdenAI(
    callbacks=[StreamingStdOutCallbackHandler()],
    feature="text",
    provider="openai",
    temperature=0.2,
    max_tokens=250,
)
prompt = """
User: Answer the following yes/no question by reasoning step by step. Can a dog drive a car?
Assistant:
"""
print(llm.invoke(prompt))

## Chaining Calls

In [ ]:
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain_core.prompts import PromptTemplate

In [ ]:
llm = EdenAI(feature="text", provider="openai", temperature=0.2, max_tokens=250)
text2image = EdenAI(feature="image", provider="openai", resolution="512x512")

In [ ]:
prompt = PromptTemplate(
    input_variables=["product"],
    template="What is a good name for a company that makes {product}?",
)

chain = LLMChain(llm=llm, prompt=prompt)

In [ ]:
second_prompt = PromptTemplate(
    input_variables=["company_name"],
    template="Write a description of a logo for this company: {company_name}, the logo should not contain text at all ",
)
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [ ]:
third_prompt = PromptTemplate(
    input_variables=["company_logo_description"],
    template="{company_logo_description}",
)
chain_three = LLMChain(llm=text2image, prompt=third_prompt)

In [ ]:
# Run the chain specifying only the input variable for the first chain.
overall_chain = SimpleSequentialChain(
    chains=[chain, chain_two, chain_three], verbose=True
)
output = overall_chain.run("hats")

In [ ]:
# print the image
print_base64_image(output)